#### Importar datos

In [110]:
import pandas as pd
import xarray as xr
import numpy as np

# 1. Load and parse dates
df = pd.read_csv(
    "/home/germankux/Downloads/temp_promedio_bogota_2010_2022.csv",
    parse_dates=["FechaObservacionHour"]
)


#### Transformar la variable FechaObservacionHour a un formato de fecha y hora

In [111]:
df["FechaObservacionHour"] = pd.to_datetime(df["FechaObservacionHour"])
# make it tz‑naive again
df["FechaObservacionHour"] = df["FechaObservacionHour"].dt.tz_localize(None)


#### Generar las variables de fecha y hora

In [112]:
# 2. Extract date and hour-of-day
df["date"] = df["FechaObservacionHour"].dt.normalize()   # midnight of each day
df["hour"] = df["FechaObservacionHour"].dt.hour          # 0–23

#### Genero 3 variables que van a representar los 3 ejes del tensor 

In [113]:
# 3. Ensure every (date, hour, station) exists
dates    = pd.date_range(df["date"].min(), df["date"].max(), freq="D")
hours    = np.arange(24)
stations = df["NombreEstacion"].unique()

#### Genero un multindice con todas las combinaciones posibles de las 3 variables de índice

In [114]:
# build the full grid
full_idx = pd.MultiIndex.from_product(
    [dates, hours, stations],
    names=["date","hour","NombreEstacion"]
)

#### Genera un pandas series de la variable de interés "temperatura" con el índice completo "full_idx"

In [115]:
# re‑index onto the full grid, pulling in your temperature values
s = ( df
      .set_index(["date","hour","NombreEstacion"])["ValorPromedioHora"]
      .reindex(full_idx)
)

In [116]:
type(s)  # should be a Series with a MultiIndex

pandas.core.series.Series

In [117]:
s.head(200).tail(34) # Ejemplo de valores que toma la variable s

date        hour  NombreEstacion                            
2010-01-01  4     EL DESTINO  - AUT                              NaN
                  LA ESMERALDA-  - AUT                           NaN
                  EL SOCHE  - AUT                                NaN
                  UNIVERSIDAD SANTO TOMÁS  - AUT                 NaN
            5     IDEAM BOGOTA  - AUT                           11.2
                  CIUDAD BOLIVAR                                 8.2
                  VILLA TERESA - AUT                             5.4
                  NUEVA GENERACION  - AUT                        NaN
                  CERRO GUADALUPE - FOPAE                        NaN
                  UAN SEDE CIRCUNVALAR - FOPAE                   NaN
                  ESC LA UNION - FOPAE                           NaN
                  UNIVERSIDAD NACIONAL  - AUT                    NaN
                  UAN SEDE USME - FOPAE                          NaN
                  SAN BENITO - FOPAE      

#### Transforma el pandas series "s" a un xarray DataArray "da"

In [118]:
# 4. Convert to xarray DataArray
da = s.to_xarray().rename("t2m")  
# da dims are ('date','hour','NombreEstacion')

#### Se realiza la interpolación de los valores faltantes en el xarray DataArray

In [119]:
# 5. Interpolate missing values “smartly”:
#   a) first along the hour‑of‑day (within each date & station) – linear in hour
da = da.interpolate_na(dim="hour", method="nearest", fill_value="extrapolate")

#   b) then along the date (within each hour & station) – linear in time
da = da.interpolate_na(dim="date", method="nearest", fill_value="extrapolate")

In [120]:
da # Como se visualiza el tensor de temperatura

<xarray.DataArray 't2m' (date: 4384, hour: 24, NombreEstacion: 34)> Size: 29MB
array([[[16.6       , 17.04159091,  9.26666667, ..., 16.8695    ,
          5.4       , 12.4375    ],
        [16.6       , 17.04159091,  9.26666667, ..., 16.8695    ,
          5.4       , 12.4375    ],
        [16.6       , 17.04159091,  9.26666667, ..., 16.8695    ,
          5.4       , 12.4375    ],
        ...,
        [18.        , 16.88701754, 13.29166667, ..., 16.13266667,
         12.9       , 12.4375    ],
        [18.2       , 16.7545    , 11.6       , ..., 15.1525    ,
         11.6       , 12.4375    ],
        [15.5       , 15.03816667, 10.475     , ..., 13.9385    ,
          9.2       , 12.99166667]],

       [[16.6       , 17.04159091,  9.26666667, ..., 16.8695    ,
          7.5       , 12.4375    ],
        [16.6       , 17.04159091,  9.26666667, ..., 16.8695    ,
          5.9       , 12.4375    ],
        [16.6       , 17.04159091,  9.26666667, ..., 16.8695    ,
          5.7       , 12.4375    ],
...
        [17.3       , 18.92191489,  7.63333333, ..., 18.384     ,
         13.1       , 13.75      ],
        [16.4       , 18.92191489,  7.63333333, ..., 18.384     ,
         12.2       , 13.75      ],
        [15.5       , 18.92191489,  7.63333333, ..., 18.384     ,
          9.8       , 13.75      ]],

       [[15.        , 14.04183333,  7.73333333, ..., 12.95283333,
         10.1       , 10.58      ],
        [14.1       , 13.6515    ,  7.73333333, ..., 12.05366667,
         10.        ,  9.5       ],
        [14.        , 13.3315    ,  7.73333333, ..., 11.225     ,
          9.5       , 11.175     ],
        ...,
        [12.4       , 18.92191489,  7.63333333, ..., 18.384     ,
          9.        , 13.75      ],
        [12.4       , 18.92191489,  7.63333333, ..., 18.384     ,
          9.        , 13.75      ],
        [12.4       , 18.92191489,  7.63333333, ..., 18.384     ,
          9.        , 13.75      ]]])
Coordinates:
  * date            (date) datetime64[ns] 35kB 2010-01-01 ... 2022-01-01
  * hour            (hour) int64 192B 0 1 2 3 4 5 6 7 ... 17 18 19 20 21 22 23
  * NombreEstacion  (NombreEstacion) object 272B 'APTO EL DORADO - TX GPRS' ....

#### Generación del tensor de pandas

In [121]:
# 6. (Optional) get back a NumPy tensor
tensor = da.values  
# shape will be (n_dates, 24, n_stations)

In [122]:
tensor.shape

(4384, 24, 34)